In [7]:
from sklearn import linear_model
import pickle as pkl
import numpy as np

In [92]:
train_file = './kitti_data/X_train.p'
train_source = './kitti_data/sources_train.p'
val_file = './kitti_data/X_val.p'
val_source = './kitti_data/sources_val.p'

val_images = pkl.load(open(val_file, 'rb')).astype(np.float32)/255
val_sources = pkl.load(open(val_source, 'rb'))
# train_images = pkl.load(open(train_file, 'rb')).astype(np.float32)/255
# train_sources = pkl.load(open(train_source, 'rb'))



N, H, W, C = val_images.shape
print(val_images.shape)

X = W//2
Y = H//2
print(X, Y)



# print(f1_flat.shape)
# # print('train pixel', frame1[Y,X,:])
# print('output target', val_images[1,Y, X,:])
# print('flat opt', f1_flat[X+Y*H,:])
print('frame', frame1[Y,X,:])
print('flat',f1_flat[W*Y+X,:])

# pixel (Y, X) in image matrix form is pixel (W*Y + X) in flattened form

# TAKES IN FLATTENED IMAGES!
class Dataset():
    def __init__(self, images, sources, nt, W=160, H=128):
        self.W = W
        self.H = H
        self.X = images
        self.N, self.L, self.C = self.X.shape
        self.nt = nt
        self.sources = sources
        self.possible_starts = np.array([i for i in range(self.X.shape[0] - self.nt) if self.sources[i] == self.sources[i + self.nt - 1]])
    
    #def get_all_examples():
    
    def get_sequence(idx):
        '''
        idx -> value in possible_starts
        sequence -> sequence of nt frames
        '''
        
        sequence = self.X[idx:idx+self.nt,:,:]
        return sequence
    
    def get_input_frame(self,idx):
        '''
        idx -> value in possible_starts
        frame -> (nt-1)th frame: next to last prediction
        '''
        frame = self.X[idx+nt-2,:,:]
        return frame
    
    def get_target_pixel(self,idx):
        '''
        idx -> value in possible_starts
        output -> RGB values of center pixel in last frame of sequence
        '''
        X, Y = (W//2, H//2)
        output = self.X[idx+nt-1, Y*W+X, :]
        return output
    
    def get_all_input_frames(self):
        input_imgs = np.zeros((len(self.possible_starts), self.L, self.C))
        for idx in self.possible_starts:
            input_imgs[idx,:] = self.get_input_frame(idx)
        return input_imgs
    
    def get_all_target_pixels(self):
        target_pixels = np.zeros((len(self.possible_starts), self.C))
        for idx in self.possible_starts:
            tp = self.get_target_pixel(idx)
            target_pixels[idx,:] = tp
        return target_pixels
        


(154, 128, 160, 3)
80 64
frame [ 0.28627452  0.24705882  0.30980393]
flat [ 0.28627452  0.24705882  0.30980393]


In [150]:
lm = linear_model.LinearRegression()

SPLIT_PT = 110
nt = 2

frame1 = val_images[0,:,:,:]
f1_flat = np.reshape(frame1, (W*H, C))
imgs_flat_3d = np.reshape(val_images, (N, W*H, C))

train_imgs_3d = imgs_flat[:SPLIT_PT,:,:]
test_imgs_3d = imgs_flat[SPLIT_PT:,:,:]
train_imgs_2d = np.reshape(val_images[:SPLIT_PT,:,:,:], (SPLIT_PT, W*H*C))
test_imgs_2d = np.reshape(val_images[SPLIT_PT:,:,:,:], (N-SPLIT_PT, W*H*C))

dataset = Dataset(train_imgs_3d, val_sources, nt, W=160, H=128)
train_Xs = dataset.get_all_input_frames()
train_Xs_2d = np.reshape(train_Xs, (len(dataset.possible_starts), W*H*C))
train_Ys = dataset.get_all_target_pixels()

lm.fit(train_Xs_2d, train_Ys)

test_dataset = Dataset(test_imgs_3d, val_sources[SPLIT_PT:], 2)
test_Xs = test_dataset.get_all_input_frames()
test_Xs_2d = np.reshape(test_Xs, (len(test_dataset.possible_starts),W*H*C ) )
test_Ys = test_dataset.get_all_target_pixels()

test_preds = lm.predict(test_Xs_2d)
print(np.mean(np.abs(test_Ys-test_preds)))

print(test_Xs.shape)
previous_Ys = test_Xs[:,Y*W+X,:]

print(np.mean(np.abs(test_Ys-previous_Ys)))
# lasso = linear_model.Lasso(0.0004, max_iter=5000)
# lasso.fit(train_Xs_2d, train_Ys)
# lasso_preds = lasso.predict(test_Xs_2d)

# print('lasso', np.mean(np.abs(test_Ys-lasso_preds)))
# print(np.count_nonzero(np.abs(lasso.coef_)))





0.197428435672
(42, 20480, 3)
0.179240585674
